In [ ]:
# Gym stuff
import gym
import gym_anytrading
# Stable baselines - rl stuff
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import A2C, PPO

# Processing libraries
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from trader import trader_agent
import cryptomanager as cm
test = trader_agent()

In [ ]:
df = cm.historical_data_df(days_delta=10000)
print(df.dtypes)
df

In [ ]:
# df = pd.read_csv("data/gmedata.csv")
env = gym.make('forex-v0', df=df, frame_bound=(5,len(df)), window_size=5)
df

In [ ]:
# Examining the model

state = env.reset()
while True: 
    # add a bit of randomness
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    if done: 
        print("info", info)
        break
        
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()

In [ ]:
!pip install finta

In [ ]:
from gym_anytrading.envs import StocksEnv
from finta import TA

In [ ]:
# df['OBV'] = TA.OBV(df)
df['EMA'] = TA.EMA(df, 200)
df['RSI'] = TA.RSI(df)
df.fillna(0, inplace=True)
df = df.join(TA.MACD(df))
df

In [ ]:
df.head(15)

In [ ]:
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Low','EMA',"MACD","SIGNAL", 'RSI']].to_numpy()[start:end]
    return prices, signal_features

In [ ]:

class Cypto_Env(ForexEnv):
    _process_data = add_signals
    
env2 = Cypto_Env(df=df, window_size=12, frame_bound=(12,len(df)))

In [ ]:
env2.signal_features

In [ ]:
df.head()

In [ ]:
env_maker = lambda: env2
env = DummyVecEnv([env_maker])

In [ ]:
from config import LOG_PATH, MODEL_PATH
from datetime import datetime
import os
 
model = A2C('MlpPolicy', env, verbose=1, tensorboard_log=LOG_PATH) 
model.device = "cuda"
name = str(datetime.now().strftime("%Y-%d-%m-%H-%M") )
print(name)
episodes = 50
step_count = 100000
for i in range (episodes):
    model.learn(total_timesteps=step_count, tb_log_name=f"A2C - {name}")
    model.save(f"{os.path.join(MODEL_PATH , name)} -{i + 1}.zip")

In [ ]:
from IPython.display import clear_output
from trader import  trader_agent
import time
import os
trade_me = trader_agent()
prev_min = 0
clear = lambda: os.system('cls')

# clear()
df
env = Cypto_Env(df=df, window_size=12, frame_bound=(len(df)-5,len(df)))
obs = env.reset()
while True: 
    try:
        cur_min = datetime.now().minute
        if cur_min > prev_min or cur_min == 1:
            df = cm.historical_data_df()
            df = df.loc[-5:]
            df['EMA'] = TA.EMA(df, 200)
            df['RSI'] = TA.RSI(df)
            df.fillna(0, inplace=True)
            df = df.join(TA.MACD(df))
            prev_min = cur_min
            env.df = df
            obs = env.reset()

            obs = obs[np.newaxis, ...]
            # print("1", obs)
            action, _states = model.predict(obs)
            # print(_states)
            if action == 1:
                trade_me.buy_position_at_market("BTC/USD")
                # print("b")
            else:
                trade_me.sell_position_market("BTCUSD")
                # print("s")
            obs, rewards, done, info = env.step(action)
            # print("2",obs)
            print(info)
            # clear_output()
    except:
        print("info", info)
        plt.figure(figsize=(10,6))
        plt.cla()
        env.render_all()
        plt.show()

In [ ]:
plt.figure(figsize=(15,6))
plt.cla()
env.render_all()
plt.show()